In [15]:
import pandas as pd
import numpy as np
import re
from cn_tools.data_processing import process_measurement, cut_df
import os

In [68]:
directory = '../Data/raw/VF_008_analysis_ref_centroid/'
def directory_eval(directory):
    '''Iterates through a raw directory and saves all processed csv files in another directory called preprocessed'''
    
    #define save directory
    save_directory = re.sub('raw', 'preprocessed', directory)
    #check if save directory already exists and create it in case 
    if not os.path.isdir(save_directory):
        os.makedirs(save_directory)
        
    #create list of all minkowski file folders to evaluate
    folders = [directory + folder + '/' for folder in os.listdir(directory) if '_mink_val' in folder]
    for folder in folders:
        
        #define name of csv the data is going to be saved to
        save_file = re.sub('raw', 'preprocessed', folder)
        save_file = re.sub('_mink_val/', '.csv', save_file)
        
        #check if file exists already, of not, process and save the data
        if not os.path.isfile(save_file):
            xyz_file = re.sub('_mink_val/', '.xyz', folder)
            df = process_measurement(path = folder, xyz_path = xyz_file)
            df = cut_df(df, r_th=30, z_th=50)
            df.to_csv(save_file)
            
    print(f'Saved processed csv files to {save_directory}')

In [69]:
directory_eval(directory)

Saved processed csv files to ../Data/preprocessed/VF_008_analysis_ref_centroid/
